<a href="https://colab.research.google.com/github/RATFIVE/GEOMAR-DeepLearning/blob/main/app/backend/small-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

## Clone the data from GitHub

In [2]:
import os
if not os.path.exists('GEOMAR-DeepLearning'):
    print(f'GEOMAR-DeepLearning does not exist')
    !git clone https://github.com/RATFIVE/GEOMAR-DeepLearning.git
    %cd GEOMAR-DeepLearning/app/backend
    !git pull
    !pip install -r requirements.txt
else:
    print(f'GEOMAR-DeepLearning exists')
    %cd GEOMAR-DeepLearning/app/backend
    !git pull
    !pip install -r requirements.txt

GEOMAR-DeepLearning exists
/content/GEOMAR-DeepLearning/app/backend
Already up to date.


## Import Libaries

In [3]:
# import all necessary libraries
import copy
import os
from PIL import Image
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import datetime
from utils.Copernicus import AdvancedCopernicus
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm import tqdm
# Ignore SettingWithCopyWarning:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)



# Display all columns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Import Data

In [4]:
START_DATE = '2025-02-01'
END_DATE = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
OUTPUT_FILENAME = 'output.nc'

BBOX= {
    # "min_lon":10.038345850696412,
    # "max_lon":10.365962458698567,
    # "min_lat":54.27381478077755,
    # "max_lat":54.52976525577923,

    "minimum_longitude":9.85083510071235,
    "maximum_longitude":10.926709174713364,
    "minimum_latitude":54.25206332481298,
    "maximum_latitude":54.97306793985031,

    "target_min_lon":10.156,
    "target_max_lon":10.170,
    "target_min_lat":54.354,
    "target_max_lat":54.365
    }

In [ ]:
copernicus = AdvancedCopernicus()
def load_ocean_data(variables=["bottomT", "mlotst", "siconc", "sithick", "sla", "so", "sob", "thetao", "uo", "vo", "wo"],
                    minimum_longitude=BBOX["minimum_longitude"],
                    maximum_longitude=BBOX["maximum_longitude"],
                    minimum_latitude=BBOX["minimum_latitude"],
                    maximum_latitude=BBOX["maximum_latitude"],
                    delete_file=True,
                    output_filename='output'):

    output_range = maximum_longitude - minimum_longitude
    output_filename = f'{output_filename}-{START_DATE}-{output_range}.nc'

    if os.path.exists(output_filename):
        print(f'File {output_filename} already exists')
        return output_filename

    data = copernicus.get_subset(
        dataset_id="cmems_mod_bal_phy_anfc_PT1H-i",
        dataset_version="202411",
        variables=variables,
        minimum_longitude=minimum_longitude,
        maximum_longitude=maximum_longitude,
        minimum_latitude=minimum_latitude,
        maximum_latitude=maximum_latitude,
        start_datetime=START_DATE,
        end_datetime=END_DATE,
        minimum_depth=0.5016462206840515,
        maximum_depth=0.5016462206840515,
        coordinates_selection_method="strict-inside",
        disable_progress_bar=False,
        output_filename=output_filename,
        delete_file=delete_file)

    return data.to_dataframe().reset_index()

training_data = load_ocean_data(
    variables=["bottomT", "mlotst", "siconc", "sithick", "sla", "so", "sob", "thetao", "uo", "vo", "wo"],
    minimum_longitude=BBOX["minimum_longitude"],
    maximum_longitude=BBOX["maximum_longitude"],
    minimum_latitude=BBOX["minimum_latitude"],
    maximum_latitude=BBOX["maximum_latitude"], delete_file=False, output_filename='training'
)

target_data = load_ocean_data(
    variables=["sla"],
    minimum_longitude=BBOX["target_min_lon"],
    maximum_longitude=BBOX["target_max_lon"],
    minimum_latitude=BBOX["target_min_lat"],
    maximum_latitude=BBOX["target_max_lat"], delete_file=False, output_filename='target'
)


# Check if training_data is a class str
if isinstance(training_data, str):
    # Read .nc file
    training_data = xr.open_dataset(training_data).to_dataframe().reset_index()
    print(f'Open Training Data as DataFrame')
else:
    print(f'Training Data is already a DataFrame')

if isinstance(target_data, str):
    # Read .nc file
    target_data = xr.open_dataset(target_data).to_dataframe().reset_index()
    print(f'Open Target Data as DataFrame')
else:
    print(f'Target Data is already a DataFrame')



## IDA

In [ ]:
def process_df(df):
    df = df.dropna(axis=1, how="all")
    df = df.dropna(axis=0, how="any")
    df = df[["time"] + [col for col in df.columns if col != "time"]]
    float_cols = df.select_dtypes(include=["float"]).columns
    df[float_cols] = df[float_cols].astype(np.float32)
    df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None).dt.round("h")
    df = df.reset_index(drop=True)
    return df

In [ ]:
training_data = process_df(training_data)
display(training_data.head(3))
display(training_data.info())

In [ ]:
target_data = process_df(target_data)
target_data.groupby(by=["time", 'latitude', 'longitude']).mean()
display(target_data.head(3))
display(target_data.info())
display(target_data['latitude'].unique())

In [ ]:
df_merged = pd.merge(training_data, target_data, on="time", how="inner", suffixes=("", "_target"))
display(df_merged.head(3))
display(df_merged.info())
display(df_merged.describe())

## EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for Seaborn
sns.set_style("whitegrid")

# Erstellen der Plot-Figur
plt.figure(figsize=(10, 5))

# Plot der SLA-Daten
sns.lineplot(x=df_merged["time"], y=df_merged["sla"], label="SLA")

# Plot der SLA Target-Daten
sns.lineplot(x=df_merged["time"], y=df_merged["sla_target"], label="SLA Target")

# Achsen und Titel setzen
plt.title("Sea surface height above sea levelsla [m]")
plt.xlabel("Time")
plt.ylabel("Sea Level Anomaly (SLA)")

# Legende anzeigen
plt.legend()

# Diagramm anzeigen
plt.show()


## Stationary

In [ ]:
# df_target = df_merged["sla_target"]

# rolling_mean = df_target.rolling(window = 7).mean(numeric_only=True)
# rolling_std = df_target.rolling(window = 7).std(numeric_only=True)
# plt.plot(df_target, color = 'blue', label = 'Original')
# plt.plot(rolling_mean, color = 'red', label = 'Rolling Mean')
# plt.plot(rolling_std, color = 'black', label = 'Rolling Std')
# plt.legend(loc = 'best')
# plt.title('Rolling Mean & Rolling Standard Deviation')
# plt.show()

In [ ]:

"""
from statsmodels.tsa.stattools import adfuller
def print_df_stats(series, order):
    x=series
    if (order>0):
        x=x.diff(order)[order:]
    result = adfuller(x)
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

print_df_stats(df_target, 0)
print_df_stats(df_target, 1)
"""

In [ ]:
# from statsmodels.tsa.stattools import acf, pacf

# diff_series=df_target.diff(2)[2:]

# lag_acf=acf(diff_series, nlags=7)
# lag_pacf=pacf(diff_series, nlags=7, method='ols')
# plt.figure(figsize=(20,10))
# plt.subplot(121)
# plt.plot(lag_acf)
# plt.axhline(y=0,linestyle='--',color='green')
# plt.axhline(y=-1.96/np.sqrt(len(diff_series)),linestyle='--',color='green')
# plt.axhline(y=1.96/np.sqrt(len(diff_series)),linestyle='--',color='green')
# plt.title('Autocorrelation Function')
# plt.subplot(122)
# plt.plot(lag_pacf)
# plt.axhline(y=0,linestyle='--',color='green')
# plt.axhline(y=-1.96/np.sqrt(len(diff_series)),linestyle='--',color='green')
# plt.axhline(y=1.96/np.sqrt(len(diff_series)),linestyle='--',color='green')
# plt.title('Partial Autocorrelation Function')

In [ ]:
cols = [col for col in df_merged.columns if df_merged[col].dtype in [np.float64, np.float32]]
cols

## Transform Data to 2D-Array

In [ ]:

# Assuming 'df' is your DataFrame
df = df_merged.copy()
# 1. Convert time to datetime (if not already)
df['time'] = pd.to_datetime(df['time'])

# Round latitudes and longitudes
df['latitude'] = df['latitude']
df['longitude'] = df['longitude']

# 2. Get unique time points (hourly)
unique_times = df['time'].dt.strftime('%Y-%m-%d %H:00:00').unique()

df = df.drop(columns=["sla_target", "sla", "latitude_target", "longitude_target"])
target = df_merged["sla_target"]

display(df.head(3))
display(df.info())

print(f'Number of Unique latituds: {len(df["latitude"].unique())}')
print(f'Number of Unique longitudes: {len(df["longitude"].unique())}')

# 3. Create function to map lat/lon to grid coordinates
def map_coordinates_to_grid(df):
    # Get unique latitudes and longitudes
    latitudes = df['latitude'].unique()
    longitudes = df['longitude'].unique()

    # Create a grid, map latitude/longitude to a 2D grid
    latitude_map = {lat: idx for idx, lat in enumerate(latitudes)}
    longitude_map = {lon: idx for idx, lon in enumerate(longitudes)}
    #print(latitude_map)
    return latitude_map, longitude_map, len(latitudes), len(longitudes)

# 4. Create RGB image for each hour
def create_image_for_time(df, latitude_map, longitude_map, img_height, img_width):


    # get cols wich are numerical
    cols = [col for col in df.columns if df[col].dtype in [np.float64, np.float32]]

    # Remove latitude and longitude from cols
    cols.remove('latitude')
    cols.remove('longitude')
    #print(cols)

    # Normalize the values of uo, wo, and vo to [0, 255]
    #scaler = MinMaxScaler((0, 255))
    #df[cols] = scaler.fit_transform(df[cols])

    # Initialize a 2D array for every feature
    image = np.zeros((img_height, img_width, len(cols)), dtype=np.float32)

    for _, row in df.iterrows():
        #print(row)
        #print(row['latitude'])
        lat_idx = latitude_map[row['latitude']]
        lon_idx = longitude_map[row['longitude']]

        # Assign values of the cols to the corresponding pixel
        for col in cols:
            image[lat_idx, lon_idx, cols.index(col)] = row[col]

    return image

learning_data = {}
# 5. Loop through each unique time point and create an image
for i, time_point in tqdm(enumerate(unique_times), desc='Creating 2D-Array Data', total=len(unique_times)):
    # Filter data for the given time
    time_data = df[df['time'].dt.strftime('%Y-%m-%d %H:00:00') == time_point]

    # Map coordinates to grid
    latitude_map, longitude_map, img_height, img_width = map_coordinates_to_grid(time_data)

    # Create RGB image for this timepoint
    image = create_image_for_time(time_data, latitude_map, longitude_map, img_height, img_width)

    learning_data[time_point] = np.array(image)

    """
    # Save or display the image
    plt.imshow(image)
    plt.title(f'Image for {time_point}')
    plt.axis('off')  # Turn off axis labels
    #plt.savefig(f'image_{time_point}.png')  # Save the image
    plt.show()
    """

    #print(image.shape)
    if i >= 1000:
      break

In [ ]:
print(len(learning_data))

In [ ]:
learning_data = list(learning_data.values())
learning_data = np.array(learning_data)
print(learning_data.shape)